In [4]:
import pandas as pd

def critical_path_analysis(schedule):
    # Convert Immediate Predecessors string to list
    for task in schedule:
        if isinstance(task['Immediate Predecessors'], str):
            task['Immediate Predecessors'] = task['Immediate Predecessors'].split(',')

    # Calculate Earliest Start (ES) and Earliest Finish (EF) times
    for task in schedule:
        if task['Immediate Predecessors'] == 0:
            task['ES'] = 0
            task['EF'] = task['Time Required']
        else:
            predecessors = task['Immediate Predecessors']
            if isinstance(predecessors, str):
                predecessors = [predecessors]
            task['ES'] = max([t['EF'] for t in schedule if t['Task'] in predecessors])
            task['EF'] = task['ES'] + task['Time Required']

    # Calculate Latest Start (LS) and Latest Finish (LF) times
    for task in reversed(schedule):
        if task['Task'] == 'I':  # Set LF for the last task
            task['LF'] = task['EF']
            task['LS'] = task['LF'] - task['Time Required']
        else:
            successors = [t['Task'] for t in schedule if t.get('Immediate Predecessors') and task['Task'] in t['Immediate Predecessors']]
            if not successors:
                task['LF'] = task['EF']
                task['LS'] = task['LF'] - task['Time Required']
            else:
                task['LF'] = min([t['LS'] for t in schedule if t['Task'] in successors])
                task['LS'] = task['LF'] - task['Time Required']

    # Calculate Slack
    for task in schedule:
        task['Slack'] = task['LF'] - task['EF']

    # Convert to DataFrame
    df = pd.DataFrame(schedule)

    # Extract critical path
    critical_path = df[df['Slack'] == 0]

    # Calculate project duration
    project_duration = critical_path['Time Required'].sum()

    return df.to_string(index=False), critical_path['Task'].tolist(), project_duration

def format_critical_path(critical_path):
    path_str = "Critical Path: "
    for i, task in enumerate(critical_path):
        if i != 0:
            path_str += " -> "
        path_str += task
    return path_str

# Example schedule
schedule = [
    {'Task': 'A', 'Time Required': 3, 'Immediate Predecessors': 0},
    {'Task': 'B', 'Time Required': 4, 'Immediate Predecessors': 'A'},
    {'Task': 'C', 'Time Required': 2, 'Immediate Predecessors': 'A'},
    {'Task': 'D', 'Time Required': 6, 'Immediate Predecessors': 'B,C'},
    {'Task': 'E', 'Time Required': 5, 'Immediate Predecessors': 'C'},
    {'Task': 'F', 'Time Required': 3, 'Immediate Predecessors': 'C'},
    {'Task': 'G', 'Time Required': 7, 'Immediate Predecessors': 'E'},
    {'Task': 'H', 'Time Required': 5, 'Immediate Predecessors': 'E,F'},
    {'Task': 'I', 'Time Required': 8, 'Immediate Predecessors': 'D,G,H'}
]

# Perform critical path analysis
result_df, critical_path, project_duration = critical_path_analysis(schedule)

# Display results
print("Critical Path Analysis Results:")
print(result_df)

print("\n" + format_critical_path(critical_path))
print("Project Duration:", project_duration)


Critical Path Analysis Results:
Task  Time Required Immediate Predecessors  ES  EF  LF  LS  Slack
   A              3                      0   0   3   3   0      0
   B              4                    [A]   3   7  11   7      4
   C              2                    [A]   3   5   5   3      0
   D              6                 [B, C]   7  13  17  11      4
   E              5                    [C]   5  10  10   5      0
   F              3                    [C]   5   8  12   9      4
   G              7                    [E]  10  17  17  10      0
   H              5                 [E, F]  10  15  17  12      2
   I              8              [D, G, H]  17  25  25  17      0

Critical Path: A -> C -> E -> G -> I
Project Duration: 25
